# Neural Networks Regression

- This is a supplement material for my lectures on ML. It sheds light on Python implementations of the supervised machine learning algorithms. 
- I assume you know Python syntax and how it works. If you don't, I highly recommend you to take a break and get introduced to the language before going forward with my code. 
- This material is represented as a Jupyter notebook, which you can easily download to reproduce the code and play around with it. 

## 1. Libraries

To build a model, we need 
- `pandas` library to work with panel dataframes
- `sklearn.model_selection.train_test_split` to split the dataset into train and test
- `sklearn.neural_network.MLPRegressor` to build a model

<!-- In addition, I'd like to show you `sklearn.preprocessing.StandardScaler` - a library for data normalization. -->

In [1]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

## 2. Data Load & Overview

Let's load the dataset and understand it.

In [2]:
# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/5x12/ml-cookbook/master/supplements/data/cars.csv')

In [3]:
# Print the first 10 rows of the dataset
df.head(10)

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.000
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.000
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.000
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.000
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.000
5,6,2,audi fox,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250.000
6,7,1,audi 100ls,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710.000
7,8,1,audi 5000,gas,std,four,wagon,fwd,front,105.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,18920.000
8,9,1,audi 4000,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875.000
9,10,0,audi 5000s (diesel),gas,turbo,two,hatchback,4wd,front,99.5,...,131,mpfi,3.13,3.40,7.0,160,5500,16,22,17859.167


This Cars Dataset represents cars characteristics, along with their prices.

- `car_ID`: Unique id of each observation (Interger)
- `symboling`: Its assigned insurance risk rating, A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.(Categorical) 		
- `carCompany`: Name of car company (Categorical)		
- `fueltype`: Car fuel type i.e gas or diesel (Categorical)		
- `aspiration`: Aspiration used in a car (Categorical)		
- `doornumber`: Number of doors in a car (Categorical)		
- `carbody`: body of car (Categorical)		
- `drivewheel`: type of drive wheel (Categorical)		
- `enginelocation`: Location of car engine (Categorical)		
- `wheelbase`: Weelbase of car (Numeric)		
- `carlength`: Length of car (Numeric)		
- `carwidth`: Width of car (Numeric)		
- `carheight` Height of car (Numeric)		
- `curbweight` The weight of a car without occupants or baggage. (Numeric)		
- `enginetype`: Type of engine. (Categorical)		
- `cylindernumber`: Cylinder placed in the car (Categorical)		
- `enginesize`: Size of car (Numeric)		
- `fuelsystem`: Fuel system of car (Categorical)		
- `boreratio`: Boreratio of car (Numeric)		
- `stroke`: Stroke or volume inside the engine (Numeric)		
- `compressionratio`: Compression ratio of car (Numeric)		
- `horsepower`: Horsepower (Numeric)		
- `peakrpm`: Car peak rpm (Numeric)		
- `citympg`: Mileage in city (Numeric)		
- `highwaympg`: Mileage on highway (Numeric)		
- `price` (Dependent variable): Price of car in \$ (Numeric)

The main aim is to build **a model that predicts a price of a car** (`price` column) based on independent variables.

## 3. Variables

Let's split the dataset into X and y, where 
- $X$ is a set of independent variables
- $y$ is a dependent, or target variable (the last column)

To make things simpler, let's pick 3 independent variables.

In [4]:
# Filter out target column
X = df[['symboling', 'enginesize', 'horsepower']]

# Print X
X

,symboling,enginesize,horsepower
0,3,130,111
1,3,130,111
2,1,152,154
3,2,109,102
4,2,136,115
...,...,...,...
200,-1,141,114
201,-1,141,160
202,-1,173,134
203,-1,145,106


In [5]:
# Select target column
y = df['price']

# Print y
y

0      13495.0
1      16500.0
2      16500.0
3      13950.0
4      17450.0
        ...   
200    16845.0
201    19045.0
202    21485.0
203    22470.0
204    22625.0
Name: price, Length: 205, dtype: float64

## 4. Model

In this section we are going to 
- build a neural nets model, 
- evaluate its accuracy, and 
- make a prediction

### 4.1. Building the model

Let's recall three simple steps:

- Split the $X$ & $y$ variables into train and test sets
- Initialize the model
- Train the model with the varialbes

In [6]:
# Split variables into train and test
X_train, X_test, y_train, y_test = train_test_split(X, #independent variables
                                                    y, #dependent variable
                                                    random_state = 3
                                                   )

In [9]:
# Initialize the model
reg = MLPRegressor(hidden_layer_sizes = [100, 100], 
                   alpha = 5.0, 
                   random_state = 3
                  )

In [10]:
# Train the model
reg.fit(X_train, y_train)

MLPRegressor(alpha=5.0, hidden_layer_sizes=[100, 100], random_state=3)

### 4.2. Checking models accuracy

After the model has trained with the data, it's essential to understand how precisely it predicts car prices. For that, we need to check model's accuracy. 

In [11]:
print(f'Accuracy (R-squared score) of NN regression model on training set: {reg.score(X_train, y_train):.2f}')
print(f'Accuracy (R-squared score) of NN regression model on test set: {reg.score(X_test, y_test):.2f}')

Accuracy (R-squared score) of NN regression model on training set: 0.67
Accuracy (R-squared score) of NN regression model on test set: 0.71


### 4.3. Making a prediction

Now that we know the model is accurate enough, we can predict the price of any car by passing independent varialbes `symboling`, `enginesize`, `horsepower` to the model. 


The method `predict` returns such a prediction.

In [12]:
reg.predict([[0, 180, 249]])

/Users/andrewwolf/.pyenv/versions/3.10.7/lib/python3.10/site-packages/sklearn/base.py:409: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(


array([25421.43933963])

## 5. Hyperparameters Tuning

Hyperparameter Tuning is one of the critical steps in model building. It helps in increasing model's accuracy even further.

### 5.1. Playing around with activation functions and regularization parameter alpha

In [16]:
# Accuracy with different activation functions and regularization parameter alpha

for this_activation in ['tanh', 'relu']:
    for this_alpha in [0.0001, 1.0, 100]:
        
        # Initialize the model
        reg = MLPRegressor(hidden_layer_sizes = [100,100],
                             activation = this_activation,
                             alpha = this_alpha)
        
        # Train the model
        reg.fit(X_train, y_train)
        
        # Print results
        print(f'''activation funtion = {this_activation} || alpha = {this_alpha}
              r-squared training: {reg.score(X_train, y_train):.2f},
              r-squared test: {reg.score(X_test, y_test):.2f} \n''')

/Users/andrewwolf/.pyenv/versions/3.10.7/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


activation funtion = tanh || alpha = 0.0001
              r-squared training: -2.83,
              r-squared test: -2.64 



/Users/andrewwolf/.pyenv/versions/3.10.7/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


activation funtion = tanh || alpha = 1.0
              r-squared training: -2.83,
              r-squared test: -2.64 



/Users/andrewwolf/.pyenv/versions/3.10.7/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/andrewwolf/.pyenv/versions/3.10.7/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


activation funtion = tanh || alpha = 100
              r-squared training: -2.83,
              r-squared test: -2.64 

activation funtion = relu || alpha = 0.0001
              r-squared training: 0.65,
              r-squared test: 0.65 

activation funtion = relu || alpha = 1.0
              r-squared training: 0.67,
              r-squared test: 0.68 

activation funtion = relu || alpha = 100
              r-squared training: 0.67,
              r-squared test: 0.71 



/Users/andrewwolf/.pyenv/versions/3.10.7/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/andrewwolf/.pyenv/versions/3.10.7/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


### 5.2. All Hyperparameters of NN Regression

There are many hyperparameters in Ridge Regression. They are all presented below  - as you can see, they have values by default. Feel free to play around with them. To learn more about hyperparameters and their meaning, visit https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.

In [17]:
reg = MLPRegressor(hidden_layer_sizes=(100,), 
                   activation='relu',
                   solver='adam', 
                   alpha=0.0001, 
                   batch_size='auto', 
                   learning_rate='constant', 
                   learning_rate_init=0.001, 
                   power_t=0.5, 
                   max_iter=200, 
                   shuffle=True, 
                   random_state=None, 
                   tol=0.0001, 
                   verbose=False, 
                   warm_start=False, 
                   momentum=0.9, 
                   nesterovs_momentum=True, 
                   early_stopping=False, 
                   validation_fraction=0.1, 
                   beta_1=0.9, 
                   beta_2=0.999, 
                   epsilon=1e-08, 
                   n_iter_no_change=10, 
                   max_fun=15000
                  )

## 6. NN Regression with normalized data

To do

In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor

#initialize scaler
scaler = MinMaxScaler()

#transform variables
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#initialize the model
clf = MLPRegressor(hidden_layer_sizes = [100, 100], alpha = 5.0, random_state = 3)

#train the model
reg.fit(X_train_scaled, y_train)

#print results
print(f'Accuracy of NN regressor on training set: {reg.score(X_train_scaled, y_train):.2f}')
print(f'Accuracy of NN regressor on test set: {reg.score(X_test_scaled, y_test):.2f}')

Accuracy of NN regressor on training set: -2.84
Accuracy of NN regressor on test set: -2.64


/Users/andrewwolf/.pyenv/versions/3.10.7/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
